In [ ]:
import datetime
import weatherMethods
import timeMethods
from imp import reload
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tools import press, proxilvl, g
import pickle
from sklearn.neighbors import KDTree, BallTree, NearestNeighbors
from scipy.spatial import cKDTree
%matplotlib inline

# Initialize Grid System & Construct lvl-specific KDTree

In [ ]:
departureTime = datetime.datetime(2013, 1, 1, 1, 30)
arrivalTime = datetime.datetime(2013, 1, 1, 6, 30)
WindClass = weatherMethods.GetWindSpeed(departureTime, arrivalTime)
Winds = WindClass.winds

In [ ]:
Tree_Dict = {}
i = 0
for levels in WindClass.lvls.keys():
    i += 1
    Grid_lat, Grid_lon = Winds[0][0][WindClass.lvls[levels]].data(22, 52, -130, -60)[1:3]
#     print(Grid.shape) 145999 * 2
    Grid = np.dstack((Grid_lat, Grid_lon))[0]
    Tree_Dict[levels] = cKDTree(Grid)
pickle.dump(Tree_Dict, open('Grid_KDTree.p','wb'))

# Map Trajectories

In [ ]:
VTrackPath = os.getcwd() + '/TFMS_NEW/New_IAHBOS2013.csv'
VTrack = pd.read_csv(VTrackPath, parse_dates=[6])
LabelData = pd.read_csv(os.getcwd() + '/TFMS_NEW/Label_IAH_BOS_2013.csv')
Sample = VTrack[VTrack.FID == 20130101632288].reset_index(drop = 1)

In [ ]:
CenterTraj = VTrack[VTrack.FID.isin(LabelData[LabelData.MedianID != -2].FID.values)].reset_index(drop = 1)

In [ ]:
st = time.time()
depTime = Sample.Elap_Time.iloc[0]
arrTime = Sample.Elap_Time.iloc[-1]

Position = Sample[['Lat','Lon','Alt']].values
Position[:,2] = Position[:,2] * 100

WindClass = weatherMethods.GetWindSpeed(depTime, arrTime)
print(time.time() - st)

uniquelvl = []

for i in range(Position.shape[0]):
    alt = Position[i,2]
    pro_lvl = proxilvl(alt,WindClass.lvls)
    Position[i,2] = pro_lvl
    if pro_lvl not in uniquelvl:
        uniquelvl.append(pro_lvl)
    else:
        pass
print(time.time() - st)

Winds = WindClass.winds
for i in range(len(uniquelvl)):
    if uniquelvl[i] >= 300:
#         31.31666667,   -94.3
#         29.95,   -95.35
#         40.76666667,   -73.6
#         42.33333333,   -70.96666667
        u_lat = 31.31666667
        b_lat = 29.95
        u_lon = -94.3
        b_lon = -95.35
        u_wind = Winds[0][0][WindClass.lvls[uniquelvl[i]]].data(lat1 = b_lat, lat2 = u_lat,lon1 = b_lon, lon2 = u_lon)
        v_wind = Winds[0][1][WindClass.lvls[uniquelvl[i]]].data(lat1 = b_lat, lat2 = u_lat,lon1 = b_lon, lon2 = u_lon)
    else:
        u_lat = 40.66666667
        b_lat = 31.45
        u_lon = -73.75
        b_lon = -94.2
        u_wind = Winds[0][0][WindClass.lvls[uniquelvl[i]]].data(lat1 = b_lat, lat2 = u_lat,lon1 = b_lon, lon2 = u_lon)
        v_wind = Winds[0][1][WindClass.lvls[uniquelvl[i]]].data(lat1 = b_lat, lat2 = u_lat,lon1 = b_lon, lon2 = u_lon)
    
    u_windspeed = u_wind[0]
    lat = u_wind[1]
    lon = u_wind[2]
    pres_lvl = np.ones(lat.shape) * uniquelvl[i]
    v_windspeed = v_wind[0]

    if i == 0:
        wind_array = np.dstack((lat, lon, pres_lvl, u_windspeed, v_windspeed))[0]

    else:
        temp_wind = np.dstack((lat, lon, pres_lvl, u_windspeed, v_windspeed))[0]
        wind_array = np.append(wind_array, temp_wind, axis = 0)

print(time.time() - st)
# tree = cKDTree(wind_array[:,:3], leafsize = 100)
# print(time.time() - st)
# _, near_idx = tree.query(Position[:,:3], k = 1)
# wind_vector = wind_array[near_idx,3:]
# print(time.time() - st)